<a href="https://colab.research.google.com/github/reemchaaban/game_system/blob/main/data-processing/player_count_dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Preparation

In [ ]:
import pandas as pd
import requests
import json
import time
from datetime import datetime, timedelta
from google.colab import drive, userdata, files

### Import all games from game library file

In [ ]:
df = pd.read_csv("game_library_data.csv")

## Get player count since start date

### JSON file containing API respones from SteamDB of player count data per game over time


In [ ]:
with open("steamdb_api_responses.json", "r") as f:
    api_responses = json.load(f)

### Based on SteamDB data, pick

In [ ]:
# calculate earliest date no earlier than 2 years from today (ensure sufficient data)
two_years_ago = datetime.today() - timedelta(days=2 * 365)
earliest_valid_date = min(
    [datetime.utcfromtimestamp(data['data']['start']) for data in api_responses.values() if data.get('success')],
    default=two_years_ago  # fallback if no valid data
)

# ensure start date is at least 2 years ago
start_date = max(earliest_valid_date, two_years_ago)

# formatting
start_date = start_date.replace(hour=0, minute=0, second=0, microsecond=0)

# list to store game data
game_data = []

# loop through each game in API responses JSON
for game_id, data in api_responses.items():
    if data.get('success'):
        release_timestamp = data['data']['start']
        release_date = datetime.utcfromtimestamp(release_timestamp).strftime("%Y-%m-%d")  # Convert to readable date

        # append to list
        game_data.append({"game_id": game_id, "game_name": f"Game {game_id}", "release_date": release_date})

game_df = pd.DataFrame(game_data)

# convert release_date to datetime format (for sorting)
game_df["release_date"] = pd.to_datetime(game_df["release_date"])

# sort by most recent release date
game_df = game_df.sort_values(by="release_date", ascending=False)

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
print(game_df)

    game_id     game_name release_date
97  2358720  Game 2358720   2024-08-20
99  1623730  Game 1623730   2024-01-19
49  1966720  Game 1966720   2023-10-23
84  1326470  Game 1326470   2023-02-23
44   990080   Game 990080   2023-02-08
5   1938090  Game 1938090   2022-10-20
95  1811260  Game 1811260   2022-09-29
12  1245620  Game 1245620   2022-02-24
50  1517290  Game 1517290   2021-11-19
51  1240440  Game 1240440   2021-11-15
4   1063730  Game 1063730   2021-09-27
29  1203220  Game 1203220   2021-08-12
9   1599340  Game 1599340   2021-06-11
22   892970   Game 892970   2021-02-02
58  1468810  Game 1468810   2021-01-27
23  1091500  Game 1091500   2020-12-10
3   1172470  Game 1172470   2020-11-05
98  1086940  Game 1086940   2020-10-06
43   739630   Game 739630   2020-09-18
35  1097150  Game 1097150   2020-08-04
70  1222670  Game 1222670   2020-06-18
87  1238810  Game 1238810   2020-06-11
61  1174180  Game 1174180   2019-12-05
19  1085660  Game 1085660   2019-10-01
90  1089350  Game 1089350

In [ ]:
# find game IDs of games released LESS than 2 years ago
recent_game_ids = game_df[game_df["release_date"] > pd.to_datetime(start_date)]["game_id"].astype(str).tolist()

# remove those recent games
df = df[~df["game_id"].astype(str).isin(recent_game_ids)]

print(f"Removed {len(recent_game_ids)} games released after {start_date}.")

Removed 3 games released after 2023-03-17 00:00:00.


In [ ]:
all_game_data = {}
player_count_history = pd.DataFrame()

# Loop through each game in the api_responses dictionary
for game_id, data in api_responses.items():
    print(f"Processing game ID: {game_id}")

    if data == {}:
      print(f"Missing data from the JSON for game ID {game_id} !")
      continue

    if data['success']:
        start_timestamp = data['data']['start']
        step_seconds = data['data']['step']
        player_counts = data['data']['values']

        # convert start timestamp to datetime
        game_start_date = datetime.utcfromtimestamp(start_timestamp)

        # calculate index to start from based on start_date
        if start_date >= game_start_date:
            days_diff = (start_date - game_start_date).days
            start_index = days_diff
        else:
            # if the game's data starts after goal start_date, skip this game
            print(f"Skipping game ID {game_id}, no data before {start_date}")
            continue

        # prepare list of dates starting from provided start_date
        dates = [start_date + timedelta(days=i) for i in range(len(player_counts) - start_index)]

        # trim player counts to start from goal start_date
        trimmed_player_counts = player_counts[start_index:]

        # replace "None" with 0
        trimmed_player_counts = [0 if count is None else count for count in trimmed_player_counts]

        # store trimmed data
        all_game_data[game_id] = trimmed_player_counts

# create df from data
player_count_history = pd.DataFrame(all_game_data, index=dates)
player_count_history.index.name = 'date'

# sort df by date
player_count_history.sort_index(inplace=True)

Processing game ID: 570
Processing game ID: 730
Processing game ID: 578080
Processing game ID: 1172470
Processing game ID: 1063730
Processing game ID: 1938090
Processing game ID: 271590
Processing game ID: 550
Processing game ID: 553850
Processing game ID: 1599340
Processing game ID: 304930
Processing game ID: 236390
Processing game ID: 1245620
Processing game ID: 105600
Processing game ID: 440
Processing game ID: 291550
Processing game ID: 431960
Processing game ID: 4000
Processing game ID: 359550
Processing game ID: 1085660
Processing game ID: 252490
Processing game ID: 346110
Processing game ID: 892970
Processing game ID: 1091500
Processing game ID: 218620
Processing game ID: 12210
Processing game ID: 238960
Processing game ID: 242760
Processing game ID: 322330
Processing game ID: 1203220
Processing game ID: 899770
Processing game ID: 413150
Processing game ID: 340
Processing game ID: 381210
Processing game ID: 945360
Processing game ID: 1097150
Processing game ID: 292030
Processing

In [ ]:
player_count_history["total players"] = player_count_history.sum(axis=1)

player_count_history.to_csv("player_count_history.csv")